# Daily data processing and updating

This notebook will be run once a day.
It will download the daily updated datasheet, containing amount of new cases and deaths per country due to the COVID-19 virus, courtesy of https://ourworldindata.org/coronavirus-source-data

After this it will convert the data to a few dataframes that I will be working with.
Namely for for each country and date that there is information about I will 4 dataframes each containing the following attributes: new cases, new deaths, total cases, total deaths

I will comment on all steps what is happening and regularly print what we're working with for transparency.

### Data import

First we'll have to download the datasheet.
For now this can only be done locally on my machine.

In [1]:
import urllib.request               # Calls url to downlload daily file
import datetime                     # Provides current date

try:
    today = datetime.date.today()
    url = 'https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-' + str(today) + '.xlsx'

    # Second argument in urlretrieve is the destination and filename for saving
    _ = urllib.request.urlretrieve(url, 'data/daily_data.xlsx')

except: 
    yesterday = datetime.date.today() - datetime.timedelta(days=1)
    url = 'https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-' + str(yesterday) + '.xlsx'

    # Second argument in urlretrieve is the destination and filename for saving
    _ = urllib.request.urlretrieve(url, 'data/daily_data.xlsx')

Next we'll need to open the downloaded .xlsx file as a pandas dataframe so we can easelly manipulate the data.

In [2]:
import pandas as pd                # Tool to easely manipulate data

df = pd.read_excel('data/daily_data.xlsx')
df

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
0,2020-03-29,29,3,2020,15,1,Afghanistan,AF,AFG,37172386.0
1,2020-03-28,28,3,2020,16,1,Afghanistan,AF,AFG,37172386.0
2,2020-03-27,27,3,2020,0,0,Afghanistan,AF,AFG,37172386.0
3,2020-03-26,26,3,2020,33,0,Afghanistan,AF,AFG,37172386.0
4,2020-03-25,25,3,2020,2,0,Afghanistan,AF,AFG,37172386.0
...,...,...,...,...,...,...,...,...,...,...
7510,2020-03-25,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
7511,2020-03-24,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14439018.0
7512,2020-03-23,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
7513,2020-03-22,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0


### Data processing

As a first step we will clean out th data a bit since we have quite some values that we do not need.

Considering the 'DateRep' column contains a datetime that Python can translate we will delete the unneeded time attributes, as well as saving the geoid's in a seperate dataset for when we need them.

In [3]:
df = df.drop(columns=['day', 'month', 'year', 'geoId', 'countryterritoryCode'])

df

,dateRep,cases,deaths,countriesAndTerritories,popData2018
0,2020-03-29,15,1,Afghanistan,37172386.0
1,2020-03-28,16,1,Afghanistan,37172386.0
2,2020-03-27,0,0,Afghanistan,37172386.0
3,2020-03-26,33,0,Afghanistan,37172386.0
4,2020-03-25,2,0,Afghanistan,37172386.0
...,...,...,...,...,...
7510,2020-03-25,0,0,Zimbabwe,14439018.0
7511,2020-03-24,0,1,Zimbabwe,14439018.0
7512,2020-03-23,0,0,Zimbabwe,14439018.0
7513,2020-03-22,1,0,Zimbabwe,14439018.0


Next, just to make our next step a little bit easier we will create two lists. One for every date, ordered from old to recent, and another for every country that can be found in the database.

In [4]:
dates = []
countries = []

for entry in df['dateRep']:
    if entry not in dates:
        dates.append(entry)

for entry in df['countriesAndTerritories']:
    if entry not in countries:
        countries.append(entry)
        
dates.sort()

Now for the actual data extraction. 
We will start with the new cases and deaths and continue from there.

To create the structures of our dataframes we will make a list filled with tuples, which in turn contain the name of the columns we will create (in this case the 'Date' and all country names) followed by a list that holds the values of cases or deaths of that country. 
We will always follow the order of the dates list we created before when filling the lists of country values to they all line up.

At the end we convert those structure lists into dictionaries to be able to use pandas on them and convert them into dataframes.

In [5]:
struct_cases = [('Date', dates)]
struct_deaths = [('Date', dates)]
struct_populations = [('Countries', []),('Populations', [])]
i = 1

for country in countries:
    country_pop = df.loc[df['countriesAndTerritories']==country]['popData2018'].values[0]
    struct_populations[0][1].append(country)
    struct_populations[1][1].append(country_pop)
    
    struct_cases.append((country, []))
    struct_deaths.append((country, []))
    
    country_df = df.loc[df['countriesAndTerritories']==country]
    collected_dates = country_df['dateRep'].tolist()
    
    for date in dates:
        if date in collected_dates:
            entry = country_df.loc[country_df['dateRep']==date]
            struct_cases[i][1].append(entry['cases'].values[0])
            struct_deaths[i][1].append(entry['deaths'].values[0])
            
        else:
            struct_cases[i][1].append(0)
            struct_deaths[i][1].append(0)
    
    i += 1

dict_cases = {title: content for (title, content) in struct_cases}
dict_deaths = {title: content for (title, content) in struct_deaths}
dict_populations = {title: content for (title, content) in struct_populations}

New we're finlly ready to create the dataframes for both new cases as well as new deaths for the global data.

In [6]:
df_cases_new = pd.DataFrame(dict_cases).set_index('Date')
df_cases_new = df_cases_new.fillna(0)
df_cases_new.name = 'New Cases'

df_cases_new

,Afghanistan,Angola,Albania,Andorra,Antigua_and_Barbuda,Algeria,Anguilla,Argentina,Armenia,Aruba,...,United_Kingdom,United_Republic_of_Tanzania,United_States_of_America,United_States_Virgin_Islands,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-25,2,0,23,31,2,42,0,86,71,5,...,1427,0,8789,0,27,6,7,11,0,0
2020-03-26,33,0,23,24,0,33,0,115,25,2,...,1452,0,13963,0,28,10,15,14,9,1
2020-03-27,0,1,28,36,4,41,2,87,39,9,...,2129,1,16797,0,21,18,1,5,2,0


In [7]:
df_deaths_new = pd.DataFrame(dict_deaths).set_index('Date')
df_deaths_new = df_deaths_new.fillna(0)
df_deaths_new.name = 'New Deaths'

df_deaths_new

,Afghanistan,Angola,Albania,Andorra,Antigua_and_Barbuda,Algeria,Anguilla,Argentina,Armenia,Aruba,...,United_Kingdom,United_Republic_of_Tanzania,United_States_of_America,United_States_Virgin_Islands,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-25,0,0,1,0,0,0,0,2,0,0,...,87,0,211,0,0,0,0,0,0,0
2020-03-26,0,0,0,0,0,0,0,2,0,0,...,41,0,249,0,0,0,0,0,0,0
2020-03-27,0,0,1,3,0,4,0,4,1,0,...,115,0,246,0,0,0,1,0,0,0


In [8]:
df_populations = pd.DataFrame(dict_populations).set_index('Countries')
df_populations = df_populations.fillna(0)
df_populations.name = 'Country populations 2018'

df_populations

,Populations
Countries,
Afghanistan,37172386.0
Angola,30809762.0
Albania,2866376.0
Andorra,77006.0
Antigua_and_Barbuda,96286.0
...,...
Uzbekistan,32955400.0
Venezuela,28870195.0
Vietnam,95540395.0


Now the last thing that remains is to use the dataframes that we just created to create the dataframes that contain the total amount of cases and deaths.

In [9]:
df_cases_total = df_cases_new.copy()
df_cases_total.name = 'Total Cases'

for country in countries:
    count = 0
    
    for date in dates:
        count += df_cases_total.at[date, country]
        df_cases_total.at[date, country] = count
        
df_cases_total

,Afghanistan,Angola,Albania,Andorra,Antigua_and_Barbuda,Algeria,Anguilla,Argentina,Armenia,Aruba,...,United_Kingdom,United_Republic_of_Tanzania,United_States_of_America,United_States_Virgin_Islands,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-25,42,2,123,164,3,231,0,387,265,17,...,8077,12,55231,17,189,55,91,134,3,2
2020-03-26,75,2,146,188,3,264,0,502,290,19,...,9529,12,69194,17,217,65,106,148,12,3
2020-03-27,75,3,174,224,7,305,2,589,329,28,...,11658,13,85991,17,238,83,107,153,14,3


In [10]:
df_deaths_total = df_deaths_new.copy()
df_deaths_total.name = 'Total Deaths'

for country in countries:
    count = 0
    
    for date in dates:
        count += df_deaths_total.at[date, country]
        df_deaths_total.at[date, country] = count
        
df_deaths_total

,Afghanistan,Angola,Albania,Andorra,Antigua_and_Barbuda,Algeria,Anguilla,Argentina,Armenia,Aruba,...,United_Kingdom,United_Republic_of_Tanzania,United_States_of_America,United_States_Virgin_Islands,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-25,1,0,5,0,0,17,0,6,0,0,...,422,0,801,0,0,0,0,0,0,1
2020-03-26,1,0,5,0,0,17,0,8,0,0,...,463,0,1050,0,0,0,0,0,0,1
2020-03-27,1,0,6,3,0,21,0,12,1,0,...,578,0,1296,0,0,0,1,0,0,1


### Combining data

To create the last datasets in this work sheet we will make a se containg the relative cases, meaning amount of cases over country population, and relative deaths, meaning amount of deaths over amount of cases.

Note that for now there are some 15 countries where the population data is missing.
This will resolved later by using data from gapminder.org, though for now we will eliminate these entries.

In [11]:
struct_cases_rel = [('Date', df_cases_total.index)]
struct_deaths_rel = [('Date', df_deaths_total.index)]

i = 1
j = 1

for country in df_cases_total.columns:
    if df_populations.at[country, 'Populations'] != 0:
        struct_cases_rel.append((country, []))

        for date in df_deaths_total.index:

                rel_value = df_cases_total.at[date, country] / df_populations.at[country, 'Populations']
                struct_cases_rel[i][1].append(rel_value)
            
        i +=1
        
    struct_deaths_rel.append((country, []))
    
    for date in df_deaths_total.index:
        if df_cases_total.at[date, country] != 0:
            rel_value = df_deaths_total.at[date, country] / df_cases_total.at[date, country]
            struct_deaths_rel[j][1].append(rel_value)
            
        else:
            struct_deaths_rel[j][1].append(0)
            
    j +=1

    
dict_cases_rel = {title: content for (title,content) in struct_cases_rel}
dict_deaths_rel = {title: content for (title,content) in struct_deaths_rel}

To finish with we will create one last data set to work with. Namely one containing the relative death rates, meaning total deaths over total cases.

In [12]:
df_cases_relative = pd.DataFrame(dict_cases_rel).set_index('Date')
df_cases_relative.name = 'Cases over population size'

df_cases_relative

,Afghanistan,Angola,Albania,Andorra,Antigua_and_Barbuda,Algeria,Argentina,Armenia,Aruba,Australia,...,United_Kingdom,United_Republic_of_Tanzania,United_States_of_America,United_States_Virgin_Islands,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
2020-01-01,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
2020-01-02,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
2020-01-03,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
2020-01-04,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-25,0.000001,6.491449e-08,0.000043,0.002130,0.000031,0.000005,0.000009,0.000090,0.000161,0.000097,...,0.000121,2.130744e-07,0.000169,0.000159,0.000055,0.000002,0.000003,0.000001,1.728925e-07,1.385136e-07
2020-03-26,0.000002,6.491449e-08,0.000051,0.002441,0.000031,0.000006,0.000011,0.000098,0.000180,0.000112,...,0.000143,2.130744e-07,0.000211,0.000159,0.000063,0.000002,0.000004,0.000002,6.915700e-07,2.077704e-07
2020-03-27,0.000002,9.737174e-08,0.000061,0.002909,0.000073,0.000007,0.000013,0.000111,0.000265,0.000127,...,0.000175,2.308306e-07,0.000263,0.000159,0.000069,0.000003,0.000004,0.000002,8.068317e-07,2.077704e-07


In [13]:
df_deaths_relative = pd.DataFrame(dict_deaths_rel).set_index('Date')
df_deaths_relative.name = 'Deaths over cases'


df_deaths_relative

,Afghanistan,Angola,Albania,Andorra,Antigua_and_Barbuda,Algeria,Anguilla,Argentina,Armenia,Aruba,...,United_Kingdom,United_Republic_of_Tanzania,United_States_of_America,United_States_Virgin_Islands,Uruguay,Uzbekistan,Venezuela,Vietnam,Zambia,Zimbabwe
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
2020-01-01,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
2020-01-02,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
2020-01-03,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
2020-01-04,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-25,0.023810,0.0,0.040650,0.000000,0.0,0.073593,0.0,0.015504,0.000000,0.0,...,0.052247,0.000000,0.014503,0.0,0.0,0.000000,0.000000,0.0,0.0,0.500000
2020-03-26,0.013333,0.0,0.034247,0.000000,0.0,0.064394,0.0,0.015936,0.000000,0.0,...,0.048589,0.000000,0.015175,0.0,0.0,0.000000,0.000000,0.0,0.0,0.333333
2020-03-27,0.013333,0.0,0.034483,0.013393,0.0,0.068852,0.0,0.020374,0.003040,0.0,...,0.049580,0.000000,0.015071,0.0,0.0,0.000000,0.009346,0.0,0.0,0.333333


The last set that we would be interested is the one tha holds the dates where, for each counry seperatelly, the countries achieved 100 cases and/or 10 deaths for the first time.

In [14]:
struct_threshold = [('ind', ['100_cases', '10_deaths'])]
i = 1

for country in df_cases_total.columns:
    cases_start = df_cases_total.loc[df_cases_total[country] > 100]
    
    if len(cases_start > 0):
        struct_threshold.append((country, []))
        
        cases_start = cases_start.index[0].date()
        struct_threshold[i][1].append(cases_start)
    
        deaths_start = df_cases_total.loc[df_deaths_total[country] > 10]
        
        if len(deaths_start) > 0:
            deaths_start = deaths_start.index[0].date()
        else: 
            deaths_start = today
        struct_threshold[i][1].append(deaths_start)
        
        i += 1

d = {title:content for (title, content) in struct_threshold}
df_thresholds = pd.DataFrame(d)

df_thresholds

,ind,Afghanistan,Albania,Andorra,Algeria,Argentina,Armenia,Australia,Austria,Azerbaijan,...,Tunisia,Turkey,Ukraine,United_Arab_Emirates,United_Kingdom,United_States_of_America,Uruguay,Uzbekistan,Venezuela,Vietnam
0,100_cases,2020-03-29,2020-03-25,2020-03-23,2020-03-23,2020-03-20,2020-03-19,2020-03-11,2020-03-09,2020-03-27,...,2020-03-25,2020-03-19,2020-03-26,2020-03-18,2020-03-06,2020-03-03,2020-03-21,2020-03-28,2020-03-26,2020-03-23
1,10_deaths,2020-03-30,2020-03-30,2020-03-30,2020-03-23,2020-03-27,2020-03-30,2020-03-26,2020-03-23,2020-03-30,...,2020-03-30,2020-03-22,2020-03-30,2020-03-30,2020-03-15,2020-03-05,2020-03-30,2020-03-30,2020-03-30,2020-03-30


### Saving

With all the datasets created we can now save them to a file to be used to train networks and regression models on.
It will always save on the same file so that all other applications can just rerun when the data is updated.

In [15]:
df_cases_new.to_csv('data/cases_new.csv')
df_cases_total.to_csv('data/cases_total.csv')

df_deaths_new.to_csv('data/deaths_new.csv')
df_deaths_total.to_csv('data/deaths_total.csv')

df_cases_relative.to_csv('data/cases_relative.csv')
df_deaths_relative.to_csv('data/deaths_relative.csv')

df_populations.to_csv('data/populations_2018.csv')
df_thresholds.to_csv('data/thresholds.csv')

### Final example

To show how we will be using these dataframes, besides training models and networks, we will create a function to plot the data that we have for a set of countries.

In [16]:
import my_tools as mt

countries_to_see = ['China', 'Italy', 'United_States_of_America', 'Spain', 'United_Kingdom', 'Belgium', 'Iran']

mt.plot_side_by_side(df_cases_new, df_cases_total, countries=countries_to_see)
mt.plot_side_by_side(df_deaths_new, df_deaths_total, countries=countries_to_see)
mt.plot_side_by_side(df_cases_relative, df_deaths_relative, countries=countries_to_see)

<Figure size 1700x600 with 2 Axes>

<Figure size 1700x600 with 2 Axes>

<Figure size 1700x600 with 2 Axes>